In [2]:
colab = False

if colab:
    from google.colab import drive
    drive.mount("/content/drive")

In [3]:
if colab:
    !pip install pandas
    !pip install matplotlib
    !pip install rasterio
    !pip install shapely
    !pip install descartes
    !pip install pyproj
    !pip install geopandas
    !pip install keras_unet
    !pip install h5py
    
    from IPython.display import display, HTML
    js = ('<script>function ConnectButton(){ '
            'console.log("Connect pushed"); '
            'document.querySelector("#connect").click()} '
            'setInterval(ConnectButton,3000);</script>')
    display(HTML(js))

In [2]:
from datetime import datetime
import sys
import geopandas as gpd
import pandas as pd
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
from os.path import join
import descartes
from PIL import Image
import h5py

import fiona 
import pyproj
import time

import warnings
from importlib import reload

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import keras
from keras.utils import Sequence
from keras.optimizers import Adam
from keras.models import load_model
from keras import callbacks as keras_cb 
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# see: https://github.com/karolzak/keras-unet
from keras_unet.models import satellite_unet 
from keras_unet.losses import jaccard_distance
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.utils import get_augmented

from keras import backend as K

# # prevent Tensorflow memory leakage
# K.clear_session()

#warnings.simplefilter("ignore")
print(sys.executable)

if colab:
    sys.path.append("/content/drive/My Drive/app_UNOSAT")
    #sys.path.append("/content/drive/My Drive/app_UNOSAT/src")

from src import load_data as ld
from src import preprocessing as pp
from src import prepare_for_network as pfn
from src import utils as utl

/Users/jurriaan/opt/anaconda3/envs/conda_unosatenv/bin/python


NameError: name 'colab' is not defined

In [6]:
# local paths, set your own path here
if colab:
    #path_main = '/content/drive/My Drive/app_UNOSAT'
    path_main = '/content/drive/Computers/My Macbook Pro'
else:
    path_main = '/Volumes/other/datasets_and_ML/UNOSAT_Challenge'

path_data_main = utl.make_dir(join(path_main, "data"))

#### put data in these dirs

# I choose:
# Training: Mosul, Najaf, Nasiryah
# Validation: Souleimaniye
# Test: Bagdad, Kirkouk, Samawah, Tikrit (no labels present)
path_data_local_train = join(path_data_main, 'Train_Dataset')
path_data_local_val = join(path_data_main, 'Validation_Dataset')
path_data_local_test = join(path_data_main, 'Evaluation_Dataset')

path_model =  utl.make_dir(join(path_data_main, 'model'))

# other dirs are made automatically
dir_temp_data =  utl.make_dir(join(path_data_main, 'data_temp'))
dir_temp_train = utl.make_dir(join(dir_temp_data, 'train'))
dir_temp_val = utl.make_dir(join(dir_temp_data, 'val'))
dir_temp_dev = utl.make_dir(join(dir_temp_data, 'data_dev'))
dir_temp_eval = utl.make_dir(join(dir_temp_data, 'evaluation'))
dir_temp_plots = utl.make_dir(join(dir_temp_data, 'plots_preprocessing'))

# settings preprocessing
quantile_clip_max = 0.999
size_sub_sample = (512, 512)
dtype = 'i16'
inv_stride = 2

## Deep learning

### develop (don't run for pipeline)

#### HDF5

In [ ]:
class My_Custom_Generator(Sequence):
    def __init__(self, file_hdf5_X, file_hdf5_Y, batch_size):
        self.file_hdf5_X = file_hdf5_X
        self.file_hdf5_Y = file_hdf5_Y
        self.batch_size = batch_size
     
    def __len__(self) :
        with h5py.File(self.file_hdf5_Y, 'r') as f1_Y:
            dataset_Y  = f1_Y['all_train_data_Y']
            len_dataset_Y = dataset_Y.shape[0]
        
        return (np.ceil(len_dataset_Y) / float(self.batch_size)).astype(np.int)
    
    def __getitem__(self, idx):
        with h5py.File(self.file_hdf5_X, 'r') as f1_X:
            dataset_X  = f1_X['all_train_data_X']
            batch_x = dataset_X[idx * self.batch_size : (idx+1) * self.batch_size, :, :, :]
        
        with h5py.File(self.file_hdf5_Y, 'r') as f1_Y:
            dataset_Y  = f1_Y['all_train_data_Y']
            batch_y = dataset_Y[idx * self.batch_size : (idx+1) * self.batch_size, :, :]

        return (batch_x, batch_y.reshape(batch_y.shape[0], batch_y.shape[1], batch_y.shape[2], 1))

In [11]:
file_hdf5_X = join(dir_temp_dev,  'testrun_X_split.hdf5')
file_hdf5_Y = join(dir_temp_dev,  'testrun_Y_split.hdf5')

In [12]:
my_training_batch_gen = My_Custom_Generator(file_hdf5_X, file_hdf5_Y, BS)

In [13]:
for i, arr in enumerate(my_training_batch_gen):
    print(arr[0].shape, arr[1].shape)
    if i>3:
        break

(2, 512, 512, 8) (2, 512, 512, 1)
(2, 512, 512, 8) (2, 512, 512, 1)
(2, 512, 512, 8) (2, 512, 512, 1)
(2, 512, 512, 8) (2, 512, 512, 1)
(2, 512, 512, 8) (2, 512, 512, 1)


In [26]:
my_training_batch_gen.__len__(), len(my_training_batch_gen)

(3037, 3037)

In [15]:
with h5py.File(file_hdf5_X, 'r') as f1_Y:
    dataset_Y  = f1_Y['all_train_data_X']
    print(dataset_Y.shape)
    

(6075, 512, 512, 8)


In [ ]:
model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  # metrics=["accuracy"],
                  metrics=[iou, iou_thresholded])

In [ ]:
H = model.fit(dataset_X, dataset_Y,  
                 batch_size=BS, 
                epochs=EPOCHS, 
                verbose=1)

In [12]:
file_hdf5_X = join(dir_temp_dev,  'testrun_X_split.hdf5')
file_hdf5_Y = join(dir_temp_dev,  'testrun_Y_split.hdf5')

In [13]:
with h5py.File(file_hdf5_Y, 'r') as f1_Y:
    dsety  = f1_Y['all_train_data_Y']
    dataset_Y = dsety[:100]
    
dataset_Y = dataset_Y.reshape(dataset_Y.shape[0], dataset_Y.shape[1], dataset_Y.shape[2], 1)#.copy()

with h5py.File(file_hdf5_X, 'r') as f1_X:
    dsetx = f1_X['all_train_data_X']
    dataset_X = dsetx[:100]

In [14]:
dataset_Y.shape, dataset_X.shape

((100, 512, 512, 1), (100, 512, 512, 8))

In [15]:
dataset_X_1 = dataset_X[:,:,:,0:1]#.copy()

In [14]:
np.sum(dataset_X_1), np.sum(dataset_Y)

(11473, 81156)

In [15]:
dataset_Y.shape, dataset_X_1.shape

((32, 512, 512, 1), (32, 512, 512, 1))

In [16]:
sys.getsizeof(dataset_X_1) / 10**9, sys.getsizeof(dataset_Y) / 10**9

(0.067109008, 0.067109008)

In [17]:
BS

8

In [16]:
train_datagen_aug = get_augmented(
        dataset_X,
        dataset_Y.reshape(dataset_Y.shape[0], dataset_Y.shape[1], dataset_Y.shape[2], 1),
        X_val=None,
        Y_val=None,
        batch_size=BS,
        seed=0,
        data_gen_args=dict(
            rotation_range=90,
            height_shift_range=0,
            shear_range=0,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='constant'
        )
    )

/Users/jurriaan/opt/anaconda3/envs/conda_unosatenv/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:940: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (100, 512, 512, 8) (8 channels).
  ' channels).')
/Users/jurriaan/opt/anaconda3/envs/conda_unosatenv/lib/python3.7/site-packages/keras_preprocessing/image/numpy_array_iterator.py:127: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (100, 512, 512, 8) (8 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


In [ ]:
dir_train_X_split = join(dir_temp_train_X, 'split')
dir_train_Y_split = join(dir_temp_train_Y, 'split')

dir_val_X_split = join(dir_temp_val_X, 'split')
dir_val_Y_split = join(dir_temp_val_Y, 'split')

dir_test_X_split = join(dir_temp_test_X, 'split')

In [17]:
class My_Custom_Generator(Sequence):
    def __init__(self, dir_X, dir_Y, batch_size):
        self.list_files_X = [join(dir_X, file_name) for file_name in os.listdir(dir_X)]
        self.list_files_Y = [join(dir_Y, file_name) for file_name in os.listdir(dir_Y)]
        self.batch_size = batch_size
     
    def __len__(self) :
        return (np.ceil(len(self.list_files_X) / float(self.batch_size))).astype(np.int)
    
    def __getitem__(self, idx):
        batch_x_in = self.list_files_X[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y_in = self.list_files_Y[idx * self.batch_size : (idx+1) * self.batch_size]
        
        b_x = np.array([np.load(file_name) for file_name in batch_x_in if os.path.splitext(file_name)[1] == '.npy'])
        b_y = np.array([np.load(file_name) for file_name in batch_y_in if os.path.splitext(file_name)[1] == '.npy'])
        
        return (b_x.reshape(b_x.shape[0], b_x.shape[2], b_x.shape[3], b_x.shape[1]), 
                b_y.reshape(b_y.shape[0], b_y.shape[1], b_y.shape[2], 1))


In [18]:
my_training_batch_gen = My_Custom_Generator(dir_train_X_split, dir_train_Y_split, BS)
my_validation_batch_gen = My_Custom_Generator(dir_val_X_split, dir_val_Y_split, BS)

In [104]:
my_training_batch_gen.__len__() * 32

480

In [24]:
for i, arr in enumerate(my_training_batch_gen):
    print(arr[0].shape, arr[1].shape)
    if i>10:
        break

(31, 512, 512, 8) (31, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)
(32, 512, 512, 8) (32, 512, 512, 1)


In [32]:
# callbacks
filepath =join(path_model, "weights-improvement-{epoch:02d}-{iou:.2f}.hdf5")

cb_chk = keras_cb.ModelCheckpoint(filepath, monitor='iou', verbose=1, save_best_only=True, mode='max')
#cb_proglog = keras_cb.ProgbarLogger()
cb_es = keras_cb.EarlyStopping(patience=5)
#cb_tb = keras_cb.tensorboard_v1.TensorBoard(log_dir=join(path_model, 'logs'), histogram_freq=1, batch_size=BS, 
                                            #write_graph=True, write_grads=True, write_images=True, update_freq='epoch')
callbacks_list = [cb_chk, cb_es]

In [26]:
model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  # metrics=["accuracy"],
                  metrics=[iou, iou_thresholded])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
len(my_training_batch_gen.list_files_X) // BS

In [ ]:
H = model.fit_generator(generator=my_training_batch_gen,
                        steps_per_epoch=len(my_training_batch_gen.list_files_X) // BS,
                        epochs=EPOCHS,
                        callbacks=callbacks_list,
                        verbose=2,
#                         validation_data = my_validation_batch_gen,
#                         validation_steps = len(my_validation_batch_gen.list_files_X) // BS
                        )



Epoch 1/1


In [ ]:
model_json = model.to_json()
path_json_file = join(path_model, "model_{}.json".format(utl.datetime_now), "w") 
with open(path_json_file as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(join(path_model, "model_epochs_{}.h5".format(utl.datetime_now)))

In [ ]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["iou"], label="train_iou")

plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/iou")
plt.legend(loc="lower left")
plt.savefig(join(path_model, "plot")

In [ ]:
# train_datagen = get_augmented(
#         trainX,
#         trainY,
#         X_val=None,
#         Y_val=None,
#         batch_size=BS,
#         seed=0,
#         data_gen_args=dict(
#             rotation_range=90,
#             height_shift_range=0,
#             shear_range=0,
#             horizontal_flip=True,
#             vertical_flip=True,
#             fill_mode='constant'
#         )
#     )

### pipeline

In [10]:
# params

BS = 2
EPOCHS = 2
size_sub_sample = (512, 512)

inv_stride = 2
INIT_LR = 1e-3
depth_img = 8

IMAGE_DIMS = size_sub_sample + (depth_img,)
model = satellite_unet(IMAGE_DIMS, num_layers=4)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

print(IMAGE_DIMS)

(512, 512, 8)


In [11]:
class My_Custom_Generator(Sequence):
    def __init__(self, file_hdf5_X, file_hdf5_Y, batch_size):
        self.file_hdf5_X = file_hdf5_X
        self.file_hdf5_Y = file_hdf5_Y
        self.batch_size = batch_size
     
    def __len__(self) :
        with h5py.File(self.file_hdf5_Y, 'r') as f1_Y:
            dataset_Y  = f1_Y['all_train_data_Y']
            len_dataset_Y = dataset_Y.shape[0]
        
        return (np.ceil(len_dataset_Y) / float(self.batch_size)).astype(np.int)
    
    def __getitem__(self, idx):
        with h5py.File(self.file_hdf5_X, 'r') as f1_X:
            dataset_X  = f1_X['all_train_data_X']
            batch_x = dataset_X[idx * self.batch_size : (idx+1) * self.batch_size, :, :, :]
        
        with h5py.File(self.file_hdf5_Y, 'r') as f1_Y:
            dataset_Y  = f1_Y['all_train_data_Y']
            batch_y = dataset_Y[idx * self.batch_size : (idx+1) * self.batch_size, :, :]

        return (batch_x, batch_y.reshape(batch_y.shape[0], batch_y.shape[1], batch_y.shape[2], 1))

In [12]:
file_hdf5_X = join(dir_temp_dev,  'testrun_X_split.hdf5')
file_hdf5_Y = join(dir_temp_dev,  'testrun_Y_split.hdf5')

In [13]:
my_training_batch_gen = My_Custom_Generator(file_hdf5_X, file_hdf5_Y, BS)

In [22]:
for i, arr in enumerate(my_training_batch_gen):
    print(arr[0].shape, arr[1].shape)
    if i>1:
        break
        
print(my_training_batch_gen.__len__(), len(my_training_batch_gen))

(2, 512, 512, 8) (2, 512, 512, 1)
(2, 512, 512, 8) (2, 512, 512, 1)
(2, 512, 512, 8) (2, 512, 512, 1)
3037 3037


In [14]:
model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  # metrics=["accuracy"],
                  metrics=[iou, iou_thresholded])

In [ ]:
# Load checkpoint:
if checkpoint_path is not None:
    # Load model:
    model = load_model(checkpoint_path)
    # Finding the epoch index from which we are resuming
    initial_epoch = get_init_epoch(checkpoint_path)
else:
    model = build_model_func()
    initial_epoch = 0

In [27]:
#checkpoints
filepath_chk = None
for file in os.listdir(path_model):
    filepath_chk = join(path_model, file)
if not filepath_chk:
    filepath_chk = join(path_model, "weights-improvement-{epoch:02d}-{iou:.3f}.hdf5")
    
print('checkpoint use: {}'.format(filepath_chk))

checkpoint use: /Volumes/other/datasets_and_ML/UNOSAT_Challenge/data/model/weights-improvement-02-0.05.hdf5


In [28]:
# callbacks
cb_chk = keras_cb.ModelCheckpoint(filepath_chk, monitor='iou', verbose=1, save_best_only=True, mode='max')
#cb_proglog = keras_cb.ProgbarLogger()
cb_es = keras_cb.EarlyStopping(patience=5)
cb_tb = keras_cb.TensorBoard(log_dir=join(path_model, 'tensoboard_logs'), histogram_freq=1, batch_size=BS, 
                                            write_graph=True, write_grads=True, write_images=True, 
                                            update_freq='epoch')
callbacks_list = [cb_chk, cb_es]# cb_tb]

In [22]:
H = model.fit_generator(generator=my_training_batch_gen,
                        steps_per_epoch=len(my_training_batch_gen),
                        epochs=EPOCHS,
                        callbacks=callbacks_list,
                        verbose=1,
                        #use_multiprocessing=True,
                        #workers=6, 
                        #max_queue_size=32
                        )

Epoch 1/2
   9/3037 [..............................] - ETA: 25:39:21 - loss: 0.4825 - iou: 0.0112 - iou_thresholded: 0.0176

KeyboardInterrupt: 

In [ ]:
model_json = model.to_json()
path_json_file = join(path_model, "model_{}.json".format(utl.datetime_now), "w") 
with open(path_json_file) as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(join(path_model, "model_epochs_{}.h5".format(utl.datetime_now)))

In [ ]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["iou"], label="train_iou")

plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/iou")
plt.legend(loc="lower left")
=
file_plot_train_info = join(path_model, 'info_train_{}.png'.format(utl.datetime_now()))
plt.savefig(file_plot_train_info)
